In [1]:
from utils.handle_data import get_integer_columns_and_dtype, get_cards_metadata, process_and_save_in_batches
import pandas as pd
import numpy as np
import random

In [2]:
SET = 'woe'

In [3]:
d_type = get_integer_columns_and_dtype('handled_data/draft_data_best_players_sc_'+SET+'.csv')
raw_data = pd.read_csv('handled_data/draft_data_best_players_sc_'+SET+'.csv', dtype=d_type)

In [4]:
#groupby draft_id
draft_id_group = raw_data.groupby('draft_id')

In [5]:
#remove all group with less than 42 rows
draft_id_group = draft_id_group.filter(lambda x: len(x) == 42)

In [6]:
draft_id_group.head(5)

,expansion,event_type,draft_id,draft_time,rank,event_match_wins,event_match_losses,pack_number,pick_number,pick,...,pack_card_Doubling Season,pack_card_Bitterblossom,pool_Defense of the Heart,pool_Doubling Season,pack_card_Defense of the Heart,pool_Rhystic Study,pool_Grave Pact,pack_card_Grave Pact,user_n_games_bucket,user_game_win_rate_bucket
19411,WOE,PremierDraft,0140bddfe6154f18980c18e9ee80e5dd,2023-10-15 09:31:07,mythic,1,1,0,0,Lord Skitter's Butcher,...,0,0,0,0,0,0,0,0,100,0.62
19412,WOE,PremierDraft,0140bddfe6154f18980c18e9ee80e5dd,2023-10-15 09:31:07,mythic,1,1,0,1,"Troyan, Gutsy Explorer",...,0,0,0,0,0,0,0,0,100,0.62
19413,WOE,PremierDraft,0140bddfe6154f18980c18e9ee80e5dd,2023-10-15 09:31:07,mythic,1,1,0,2,Rootrider Faun,...,0,0,0,0,0,0,0,0,100,0.62
19414,WOE,PremierDraft,0140bddfe6154f18980c18e9ee80e5dd,2023-10-15 09:31:07,mythic,1,1,0,3,Aquatic Alchemist,...,0,0,0,0,0,0,0,0,100,0.62
19415,WOE,PremierDraft,0140bddfe6154f18980c18e9ee80e5dd,2023-10-15 09:31:07,mythic,1,1,0,4,Gadwick's First Duel,...,0,0,0,0,0,0,0,0,100,0.62


In [7]:
def extract_card_names(row, prefix):
    """ Extrai os nomes das cartas com base no prefixo e retorna uma lista de nomes. """
    return [name[len(prefix):] for name, value in row.items() if name.startswith(prefix) and value]

In [8]:
# Criação das novas colunas 'pool' e 'pack'
draft_id_group['pool'] = draft_id_group.apply(lambda row: extract_card_names(row, 'pool_'), axis=1)
draft_id_group['pack'] = draft_id_group.apply(lambda row: extract_card_names(row, 'pack_card_'), axis=1)

In [9]:
filtered_df = draft_id_group[['draft_id', 'rank','pick_number', 'pack_number', 'pick','pack', 'pool']]

In [10]:
filtered_df.head(5)

,draft_id,rank,pick_number,pack_number,pick,pack,pool
19411,0140bddfe6154f18980c18e9ee80e5dd,mythic,0,0,Lord Skitter's Butcher,"[Archon's Glory, Cruel Somnophage, Disdainful ...",[]
19412,0140bddfe6154f18980c18e9ee80e5dd,mythic,1,0,"Troyan, Gutsy Explorer","[Beanstalk Wurm, Besotted Knight, Break the Sp...",[Lord Skitter's Butcher]
19413,0140bddfe6154f18980c18e9ee80e5dd,mythic,2,0,Rootrider Faun,"[Archive Dragon, Bellowing Bruiser, Besotted K...","[Lord Skitter's Butcher, Troyan, Gutsy Explorer]"
19414,0140bddfe6154f18980c18e9ee80e5dd,mythic,3,0,Aquatic Alchemist,"[Aquatic Alchemist, Brave the Wilds, Break the...","[Lord Skitter's Butcher, Rootrider Faun, Troya..."
19415,0140bddfe6154f18980c18e9ee80e5dd,mythic,4,0,Gadwick's First Duel,"[Boundary Lands Ranger, Break the Spell, Eriet...","[Aquatic Alchemist, Lord Skitter's Butcher, Ro..."


In [11]:
# Seed for reproducibility
np.random.seed(42)

# Get a list of unique group identifiers (i.e., unique 'draft_id's)
unique_ids = filtered_df['draft_id'].unique()

# Sample 20% of the unique identifiers for the test set
test_ids = np.random.choice(unique_ids, size=int(len(unique_ids) * 0.20), replace=False)

# Create a new list for the remaining ids (those not in test set)
remaining_ids = [id for id in unique_ids if id not in test_ids]

# Sample 25% of the remaining identifiers for the validation set (0.25 * 0.8 = 0.2)
val_ids = np.random.choice(remaining_ids, size=int(len(remaining_ids) * 0.25), replace=False)

# The rest (60% of total) goes to the train set
train_ids = [id for id in remaining_ids if id not in val_ids]

In [12]:
# Create the sets based on the sampled identifiers
test_set = filtered_df[filtered_df['draft_id'].isin(test_ids)]
val_set = filtered_df[filtered_df['draft_id'].isin(val_ids)]
train_set = filtered_df[filtered_df['draft_id'].isin(train_ids)]


#save data
test_set.to_csv('dataset/test_set.csv', index=False)
val_set.to_csv('dataset/val_set.csv', index=False)
train_set.to_csv('dataset/train_set.csv', index=False)

### Add metadata

In [13]:
#Load all cards data
meta_data_card = pd.read_csv('handled_data/'+SET+'_cards_data.csv')

In [14]:
meta_data_card.head(5)

,name,rarity,color_identity,cmc,colors
0,Agatha of the Vile Cauldron,mythic,M,2.0,M
1,Agatha's Champion,uncommon,G,5.0,G
2,Agatha's Soul Cauldron,mythic,T,2.0,T
3,Aquatic Alchemist // Bubble Up,common,U,2.0,U
4,Archive Dragon,uncommon,U,6.0,U


In [15]:
meta_data_card = get_cards_metadata('handled_data/'+SET+'_cards_data.csv')

In [16]:
meta_data_card.head(5)

,name,cmc,id,rarity_common,rarity_mythic,rarity_rare,rarity_uncommon,colors_B,colors_G,colors_M,colors_R,colors_T,colors_U,colors_W
0,Agatha of the Vile Cauldron,-0.578877,0,0,1,0,0,0,0,1,0,0,0,0
1,Agatha's Champion,1.357604,1,0,0,0,1,0,1,0,0,0,0,0
2,Agatha's Soul Cauldron,-0.578877,2,0,1,0,0,0,0,0,0,1,0,0
3,Aquatic Alchemist // Bubble Up,-0.578877,3,1,0,0,0,0,0,0,0,0,1,0
4,Archive Dragon,2.003098,4,0,0,0,1,0,0,0,0,0,1,0


In [17]:
#process_and_save_in_batches('dataset/train_set.csv', meta_data_card, 'dataset/train',batch_size=42000)
#process_and_save_in_batches('dataset/test_set.csv', meta_data_card, 'dataset/test',batch_size=42000)
#process_and_save_in_batches('dataset/val_set.csv', meta_data_card, 'dataset/val',batch_size=42000)
